### Summary
In contrast to the generate augmented image which put the object on filled background, this script put the object on top of black background of the same size

In [ ]:
import os
import cv2
import numpy as np
import random

In [ ]:
# To make sure that augmented images is not generated from validation object/background
VALIDATION_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Validation\CV_0'

#OBJECT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Object'
OBJECT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Object'
OUTPUT_PATH = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Segmented'

# Minimum dimension of the background, will be extended if the object size is bigger
MIN_IMAGE_SHAPE = (480,480)

In [ ]:
if os.path.isdir(OUTPUT_PATH) == False:
    os.mkdir(OUTPUT_PATH)

In [ ]:
# Set of filenames not to be used as background or foreground for each subdir
restricted_files = dict()

for subdir_name in os.listdir(VALIDATION_PATH):
    subdir_path = os.path.join(VALIDATION_PATH, subdir_name)
    restricted_files[subdir_name] = set()
    for image_name in os.listdir(subdir_path):
        restricted_files[subdir_name].add(image_name.split('.')[0])

In [ ]:
def generate_segmented_image(ob_path, out_path):
    ob = cv2.imread(ob_path, cv2.IMREAD_UNCHANGED)
    bg = np.zeros((np.max([ob.shape[0], MIN_IMAGE_SHAPE[0]]), 
                   np.max([ob.shape[1], MIN_IMAGE_SHAPE[1]]), 
                   3))
    
    # Get the top left of bounding box coordinate on the background to be replaced with the object
    # Object should be centered in the background, with possibly a little translation
    # Just naming them so they are easier to imagine
    bg_center_x = bg.shape[1] // 2
    bg_center_y = bg.shape[0] // 2
    bg_width = bg.shape[1]
    bg_height = bg.shape[0]
    ob_width = ob.shape[1]
    ob_height = ob.shape[0]
    
    # Make sure that the object is not bigger than the background in any dimension
    if ob_width > bg_width or ob_height > bg_height:
        return False

    box_topleft_x = bg_center_x - (ob_width//2)
    # Readjust the value if they go beyond the limit (0,bg_width-1) and (0,bg_height-1)
    box_topleft_x = 0 if box_topleft_x < 0 else box_topleft_x
    if box_topleft_x + ob_width > bg_width - 1:
        excess_width = (box_topleft_x + ob_width) - (bg_width - 1)
        box_topleft_x -= excess_width
    # Used to horizontally flip the object
    box_botright_x =box_topleft_x + ob_width
    
    box_topleft_y = bg_center_y - (ob_height//2)
    # Readjust the value if they go beyond the limit (0,bg_width-1) and (0,bg_height-1)
    box_topleft_y = 0 if box_topleft_y < 0 else box_topleft_y
    if box_topleft_y + ob_height > bg_height - 1:
        excess_height = (box_topleft_y + ob_height) - (bg_height - 1)
        box_topleft_y -= excess_height
        
    # Replace the background pixel with the object pixel if alpha > 0
    obj_pixels_y, obj_pixels_x = np.where(ob[:,:,3] > 0)
    for py,px in zip(obj_pixels_y, obj_pixels_x):
        bg_x = box_topleft_x+px
        bg_y = box_topleft_y+py
        bg[bg_y, bg_x] = ob[py,px,:3]
    cv2.imwrite(out_path, bg)
    del bg,ob
    return True

In [ ]:
# Start the loop from the object images
for obj_subdir_name in os.listdir(OBJECT_PATH):
    print('Augmenting',obj_subdir_name)
    obj_subdir_path = os.path.join(OBJECT_PATH, obj_subdir_name)
    output_subdir_path = os.path.join(OUTPUT_PATH, obj_subdir_name)
    if os.path.isdir(output_subdir_path) == False:
        os.mkdir(output_subdir_path)
    for obj_image_name in os.listdir(obj_subdir_path):
        # check for restricted files
        # object image follow the naming scheme of $imagename_index.png, so split based on _ instead
        obj_image_name_ori = obj_image_name.split('_')[0]
        if obj_image_name_ori in restricted_files[obj_subdir_name]:
            continue
            
        obj_image_path = os.path.join(obj_subdir_path, obj_image_name)
        # Output image name is $objname_s.jpg
        output_image_path = os.path.join(output_subdir_path, 
                                         obj_image_name_ori + '_s.jpg')
        generate_segmented_image(obj_image_path, output_image_path)